In [1]:
import numpy as np
import config
import matplotlib
import matplotlib.pyplot as plt
config.conf_matplolib()
#from spharm import Spharmt, getspecindx
from spectral.spherical.transforms import Yslm_vec, SHExpand
from spectral.spherical.grids import GLGrid
#from waveformtools.grids import GLGrid
import h5py
from waveformtools.diagnostics import method_info

/mnt/pfs/vaishak.p/Projects/Codes/custom_libraries/waveformtools/waveformtools
No module named 'pycbc'


In [2]:
from config.verbosity import levels
vl = levels()
vl.set_print_verbosity(2)

print verbosity has been set to 2 at /mnt/pfs/vaishak.p/Projects/Codes/custom_libraries/config/config/__init__.py


In [3]:
ell_max=14

gl_info = GLGrid(L=ell_max)
theta_grid, phi_grid = gl_info.meshgrid

In [4]:
from spectral.spherical.Yslm_mp import Yslm_mp

# waveformtools optimized

In [5]:
# Compute and cache SHs                 
Ylm_opt = Yslm_mp(ell_max=ell_max, 
                    spin_weight=0, 
                    theta=theta_grid, 
                    phi=phi_grid)
Ylm_opt.run()

In [6]:
type(Ylm_opt.sYlm_modes._modes_data)

numpy.ndarray

In [7]:
Ylm_opt.sYlm_modes.mode(2, 2).shape

(15, 30)

# waveformtools old

# spherepack

# spherical

In [8]:
from spectral.spherical.swsh import create_spherical_Yslm_modes_array

In [9]:
Ylm_spherical = create_spherical_Yslm_modes_array(theta=theta_grid, 
                                phi=phi_grid, 
                                ell_max=ell_max,
                                spin_weight=0)

Created a modes array of mode axis shape (15, 30)


In [10]:
ell=2
emm=-1
Ylm_spherical.mode(ell, emm)[8, 1]

(-0.14892669419269364+0.031655346058481804j)

In [36]:
from spectral.spherical.swsh import Yslm_prec, Yslm
ell=3
emm=-2
theta=np.pi/6
phi=2
Yslm_prec(ell=ell, emm=emm, theta=theta, phi=phi, spin_weight=0), Yslm(ell=ell, emm=emm, theta=theta, phi=phi, spin_weight=0)

(-0.144629335720130772596631 + 0.167454922948849377967284*I,
 (-0.14462933572013079+0.16745492294884934j))

In [11]:

for ell in range(ell_max + 1):
    emm_list = np.arange(-ell, ell + 1)
    for emm in emm_list:

        #print(f"l{ell} m{emm}")
        Ylm = Yslm_vec(
            spin_weight=0,
            emm=emm,
            ell=ell,
            theta_grid=theta_grid,
            phi_grid=phi_grid,
        )

        Ylm_opt_wft = Ylm_opt.mode(ell, emm)
        Ylm_sp = Ylm_spherical.mode(ell, emm)

        np.testing.assert_array_almost_equal(Ylm_opt_wft, Ylm, 16, "Opt must agree with vec")
        np.testing.assert_array_almost_equal(Ylm_sp, Ylm, 12, "Spherical must agree with vec")